In [1]:
# 챗봇, 번역, 이미지 캡셔닝 등에 사용되는 시퀀스 합습/생성 모델인 Seq2Seq2 를 구현
# 영어 단어를 한국어 단어로 번역하는 프로그램을 생성
import tensorflow as tf
import numpy as np

In [2]:
# S: 디코딩 입력의 시작을 나타내는 심볼
# E : 디코딩 출력의 끝을 나타내는 심볼
# P : 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼
#    예) 현재 배치 데이터의 최대 크기가 4인 경우
#       word -> ['w', ''o','r','d']
#       to   -> ['t', 'o', 'P', 'P']

char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 영어를 한글로 번역하기 위한 학습 데이터
seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'], ['girl' , '소녀'], ['kiss', '키스'], ['love', '사랑']]

In [15]:
num_dic

{'E': 1,
 'P': 2,
 'S': 0,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 '나': 31,
 '녀': 36,
 '놀': 33,
 '단': 29,
 '랑': 40,
 '무': 32,
 '사': 39,
 '소': 35,
 '스': 38,
 '어': 30,
 '이': 34,
 '키': 37}

In [16]:
dic_len

41

In [17]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in seq_data:
        # 인코더 셀의 입력값, 입력단어의 글자들을 한글자씩 떼어 배열로 만든다.
        input = [num_dic[n] for n in seq[0]]
        print(input)
        # 디코더 셀의 입력값, 시작을 나타내는 S 심볼을 맨 앞에 붙여준다.
        output = [num_dic[n] for n in ('S' + seq[1])]
        print(output)
        # 학습을 위해 비교할 디코더 셀의 출력값, 끝나는 것을 알려주기 위해 마지막에 E를 붙인다.
        target = [num_dic[n] for n in (seq[1]+'E')]
        print(target)
        
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        # 출력값만 one-hot 인코딩이 아님(sparse_softmax_cross_entropy_with_logits 사용)
        target_batch.append(target)
        
        print(input_batch , output_batch, target_batch)
    return input_batch, output_batch, target_batch

In [4]:
#######
# 옵션 설정
#######
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
# 입력과 출력의 형태가 one-hot 인코딩으로 같으므로 크기도 같다. 
n_class = n_input = dic_len 

#######
# 신경망 모델 구성
#######
# Seq2Seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같음 
# [batch_size, time steps, input size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])

# [batch_size, time steps]
targets = tf.placeholder(tf.int64, [None, None])

# 인코더 셀을 구성한다. 
with tf.variable_scope('encode'):
    enc_cell = tf.contrib.rnn.BasicRNNCell(n_hidden)
    enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)
    
# 디코더 셀을 구성한다. 
with tf.variable_scope('decode'):
    dec_cell = tf.contrib.rnn.BasicRNNCell(n_hidden)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob = 0.5)
    
    # Seq2Seq 모델은 인코더 셀의 최종 상태값을 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)
    
model = tf.layers.dense(outputs, n_class, activation = None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = model, labels = targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [6]:
#######
# 신경망 모델 학습
#######
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict= {enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    print('Epoch :', '%04d'  %(epoch+1), 'cost = ', '{:.6f}'.format(loss))

print('최적화 완료')

Epoch : 0001 cost =  3.868549
Epoch : 0002 cost =  2.881434
Epoch : 0003 cost =  1.814868
Epoch : 0004 cost =  0.924634
Epoch : 0005 cost =  0.734733
Epoch : 0006 cost =  0.443179
Epoch : 0007 cost =  0.377039
Epoch : 0008 cost =  0.322741
Epoch : 0009 cost =  0.310799
Epoch : 0010 cost =  0.221210
Epoch : 0011 cost =  0.204293
Epoch : 0012 cost =  0.213423
Epoch : 0013 cost =  0.168305
Epoch : 0014 cost =  0.120040
Epoch : 0015 cost =  0.073173
Epoch : 0016 cost =  0.176170
Epoch : 0017 cost =  0.141845
Epoch : 0018 cost =  0.030881
Epoch : 0019 cost =  0.084992
Epoch : 0020 cost =  0.080508
Epoch : 0021 cost =  0.082307
Epoch : 0022 cost =  0.014069
Epoch : 0023 cost =  0.050658
Epoch : 0024 cost =  0.008832
Epoch : 0025 cost =  0.008732
Epoch : 0026 cost =  0.014696
Epoch : 0027 cost =  0.006984
Epoch : 0028 cost =  0.018849
Epoch : 0029 cost =  0.023353
Epoch : 0030 cost =  0.008780
Epoch : 0031 cost =  0.007052
Epoch : 0032 cost =  0.002633
Epoch : 0033 cost =  0.002798
Epoch : 00

In [13]:
######
# 번역 테스트
######
# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수 
def translate(word):
    # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어] 사용하지만
    # 예측시에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미 없는 값인 P로 채운다. 
    seq_data = [word, 'P' * len(word)]
    
    input_batch, output_batch, target_batch = make_batch([seq_data])
    
    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    prediction = tf.argmax(model, 2)
    
    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    
    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다. 
    decoded = [char_arr[i] for i in result[0]]
    
    # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열을 만든다. 
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    
    return translated

In [18]:
print('\n=== 번역 테스트 ===')

print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))


=== 번역 테스트 ===
[25, 17, 20, 6]
[0, 2, 2, 2, 2]
[2, 2, 2, 2, 1]
[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
  